# Week 9-1: Unsupervised Machine Learning

## Bibliotheken importeren

In [70]:
import sqlite3
import pandas as pd
from sklearn.cluster import KMeans
import math
import matplotlib.pyplot as plt
import warnings
import numpy as np
warnings.simplefilter('ignore')

## Data inlezen en kolommen selecteren

**connectie**

In [71]:
go_sales_con = sqlite3.connect('Great_Outdoors_Data_SQLite\go_sales.sqlite')
go_crm_con = sqlite3.connect('Great_Outdoors_Data_SQLite\go_crm.sqlite')
go_staff_con = sqlite3.connect('Great_Outdoors_Data_SQLite\go_staff.sqlite')
go_inv_con = 'Great_Outdoors_Data_SQLite\GO_SALES_INVENTORY_LEVELSData.csv'
go_forecast_con = 'Great_Outdoors_Data_SQLite\GO_SALES_PRODUCT_FORECASTData.csv'

**Dataframes inlezen**

In [72]:
Go_staff_queries = {
    'Course': 'SELECT * FROM Course',
    'Training': 'SELECT * FROM Training',
    'Sales_staff': 'SELECT * FROM Sales_staff',
    'Satisfaction': 'SELECT * FROM Satisfaction',
    'Satisfaction_type': 'SELECT * FROM Satisfaction_type',
}

Go_crm_queries = {
    'Retailer_contact': 'SELECT * FROM Retailer_contact',
    'Retailer_site' : 'SELECT * FROM Retailer_site',
    'Retailer' : 'SELECT * FROM Retailer',
    'Retailer_type' : 'SELECT * FROM Retailer_type',
    'COUNTRY': 'SELECT * FROM COUNTRY',
    'Sales_territory': 'SELECT * FROM Sales_territory'
}

Go_sales_queries = {
    'Order_method': 'SELECT * FROM Order_method',
    'Order_header': 'SELECT * FROM Order_header',
    'Order_details': 'SELECT * FROM Order_details',
    'Returned_item': 'SELECT * FROM Returned_item',
    'Return_reason': 'SELECT * FROM Return_reason',
    'Product': 'SELECT * FROM Product',
    'PRODUCT_TYPE': 'SELECT * FROM PRODUCT_TYPE',
    'PRODUCT_LINE' : 'SELECT * FROM PRODUCT_LINE',
    'Sales_TARGETData' : 'SELECT * FROM Sales_TARGETData',
    'Sales_branch': 'SELECT * FROM Sales_branch'
}


dataframes = {}

# Lees elke tabel in een DataFrame
for table_name, query in Go_staff_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, go_staff_con)

for table_name, query in Go_crm_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, go_crm_con)

for table_name, query in Go_sales_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, go_sales_con)

# Lees de CSV-bestanden in pandas DataFrames
GO_SALES_INVENTORY_LEVELSData = pd.read_csv(go_inv_con, index_col=False)
GO_SALES_PRODUCT_FORECASTData = pd.read_csv(go_forecast_con, index_col=False)

# Voeg de DataFrames toe aan de dataframes dictionary
dataframes['GO_SALES_INVENTORY_LEVELSData'] = GO_SALES_INVENTORY_LEVELSData
dataframes['GO_SALES_PRODUCT_FORECASTData'] = GO_SALES_PRODUCT_FORECASTData

#als je ik elk tabel als een dataframe/ variabele wil behandelen of aanroepen moet ik dit uitvoeren.
for table_name, df in dataframes.items():
    globals()[table_name] = df

Sales_staff['DATE_HIRED'] = pd.to_datetime(Sales_staff['DATE_HIRED'], errors='coerce')
Sales_staff['DATE_HIRED'] = Sales_staff['DATE_HIRED'].dt.date

print(GO_SALES_INVENTORY_LEVELSData)

      INVENTORY_YEAR  INVENTORY_MONTH  PRODUCT_NUMBER  INVENTORY_COUNT
0               2021                4              48             1932
1               2021                4              49             1400
2               2021                4              50            21705
3               2021                4              51             9710
4               2021                4              52             5616
...              ...              ...             ...              ...
3883            2022               12             111             1128
3884            2022               12             112             7810
3885            2022               12             113             3485
3886            2022               12             114              350
3887            2022               12             115             3310

[3888 rows x 4 columns]


**Kolomen selecteren**

In [73]:
df = pd.merge(Sales_branch, COUNTRY, on = 'COUNTRY_CODE')

df = df.loc[:, ['ADDRESS1', 'ADDRESS2', 'CITY', 'REGION', 'COUNTRY_EN']]

df

,ADDRESS1,ADDRESS2,CITY,REGION,COUNTRY_EN
0,Schwabentor 35,None,Hamburg,None,Germany
1,Leopoldstraße 36,None,München,None,Germany
2,Isafjordsgatan 30 C,None,Kista,None,Sweden
3,"7800, 756 - 6th Avenue. S.W.",None,Calgary,Alberta,Canada
4,789 Yonge Street,None,Toronto,Ontario,Canada
5,1288 Dorchester Avenue,None,Boston,Massachusetts,United States
6,299 Yale Avenue,None,Seattle,Washington,United States
7,1288 South Barrington Ave.,None,Los Angeles,California,United States
8,10032 NW 186th,None,Miami,Florida,United States
9,"6c, rue de l'Église",None,Lyon,None,France


## One-hot encoding van onafhankelijke niet-numerieke variabelen

In [74]:
df.dtypes 

ADDRESS1      object
ADDRESS2      object
CITY          object
REGION        object
COUNTRY_EN    object
dtype: object

In [75]:
dummies_dataframe = pd.get_dummies(df)
dummies_dataframe 

,ADDRESS1_10032 NW 186th,ADDRESS1_1288 Dorchester Avenue,ADDRESS1_1288 South Barrington Ave.,"ADDRESS1_154, Choung-Cheng South Road",ADDRESS1_202-2-3 Hyakunincho,ADDRESS1_2315 Queen's Ave,"ADDRESS1_234-12, Kongdeok-Dong",ADDRESS1_299 Yale Avenue,ADDRESS1_3 Albany Court,ADDRESS1_543-225 Asahi,...,COUNTRY_EN_Japan,COUNTRY_EN_Korea,COUNTRY_EN_Mexico,COUNTRY_EN_Netherlands,COUNTRY_EN_Spain,COUNTRY_EN_Sweden,COUNTRY_EN_Switzerland,COUNTRY_EN_Taiwan,COUNTRY_EN_United Kingdom,COUNTRY_EN_United States
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
6,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
7,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
8,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [76]:
df = pd.concat([df, dummies_dataframe], axis = 1)
df = df.drop(['ADDRESS1', 'ADDRESS2', 'CITY', 'REGION', 'COUNTRY_EN'], axis = 1)
df.columns

Index(['ADDRESS1_10032 NW 186th', 'ADDRESS1_1288 Dorchester Avenue',
       'ADDRESS1_1288 South Barrington Ave.',
       'ADDRESS1_154, Choung-Cheng South Road', 'ADDRESS1_202-2-3 Hyakunincho',
       'ADDRESS1_2315 Queen's Ave', 'ADDRESS1_234-12, Kongdeok-Dong',
       'ADDRESS1_299 Yale Avenue', 'ADDRESS1_3 Albany Court',
       'ADDRESS1_543-225 Asahi', 'ADDRESS1_6c, rue de l'Église',
       'ADDRESS1_75, rue du Faubourg St-Honoré',
       'ADDRESS1_7800, 756 - 6th Avenue. S.W.', 'ADDRESS1_789 Yonge Street',
       'ADDRESS1_Avenida Paulista, 333', 'ADDRESS1_Birchstraße 13',
       'ADDRESS1_Interleuvenlaan 2', 'ADDRESS1_Isafjordsgatan 30 C',
       'ADDRESS1_Jedleser Straße 7', 'ADDRESS1_Kauppakatu 33',
       'ADDRESS1_Leopoldstraße 36', 'ADDRESS1_No. 1277 Beijing Xi Road',
       'ADDRESS1_Piazza Duomo, 1', 'ADDRESS1_Plaza de la Constitución, s/n',
       'ADDRESS1_Prol. Paseo de la Reforma No. 51', 'ADDRESS1_Schwabentor 35',
       'ADDRESS1_Singelgravenplein 4', 'ADDRESS1_Vict

## Clusteringmodel bouwen 

**Train de model, en geef array met cluster-indelingen**

In [77]:
kmeans = KMeans(n_clusters = 3, random_state= 42) 
prediction_results = kmeans.fit_predict(df)
prediction_results 

array([1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1])

In [78]:
df['Centrum'] = prediction_results
df

,ADDRESS1_10032 NW 186th,ADDRESS1_1288 Dorchester Avenue,ADDRESS1_1288 South Barrington Ave.,"ADDRESS1_154, Choung-Cheng South Road",ADDRESS1_202-2-3 Hyakunincho,ADDRESS1_2315 Queen's Ave,"ADDRESS1_234-12, Kongdeok-Dong",ADDRESS1_299 Yale Avenue,ADDRESS1_3 Albany Court,ADDRESS1_543-225 Asahi,...,COUNTRY_EN_Korea,COUNTRY_EN_Mexico,COUNTRY_EN_Netherlands,COUNTRY_EN_Spain,COUNTRY_EN_Sweden,COUNTRY_EN_Switzerland,COUNTRY_EN_Taiwan,COUNTRY_EN_United Kingdom,COUNTRY_EN_United States,Centrum
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,1
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,2
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,2
5,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,1
6,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,1
7,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,1
8,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,1
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1


## handmatig

**maak dataframe met berekende centra's**

In [79]:
kmeans_centra = pd.DataFrame(kmeans.cluster_centers_)
kmeans_centra

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,1.000000e+00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,...,0.08,0.04,6.938894e-18,0.04,0.04,0.04,0.04,0.04,0.08,0.16
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.000000e+00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


**De waarde vertegenwoordigen de centra van clusters gebaseerd op de combinatie van kolommen, in dit geval zijn er 3 clusters**

In [80]:
for i in range(len(kmeans_centra.columns)):
    kmeans_centra = kmeans_centra.rename(columns = {i : f'{df.columns[i]}'})
    
kmeans_centra

,ADDRESS1_10032 NW 186th,ADDRESS1_1288 Dorchester Avenue,ADDRESS1_1288 South Barrington Ave.,"ADDRESS1_154, Choung-Cheng South Road",ADDRESS1_202-2-3 Hyakunincho,ADDRESS1_2315 Queen's Ave,"ADDRESS1_234-12, Kongdeok-Dong",ADDRESS1_299 Yale Avenue,ADDRESS1_3 Albany Court,ADDRESS1_543-225 Asahi,...,COUNTRY_EN_Japan,COUNTRY_EN_Korea,COUNTRY_EN_Mexico,COUNTRY_EN_Netherlands,COUNTRY_EN_Spain,COUNTRY_EN_Sweden,COUNTRY_EN_Switzerland,COUNTRY_EN_Taiwan,COUNTRY_EN_United Kingdom,COUNTRY_EN_United States
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,1.000000e+00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,...,0.08,0.04,6.938894e-18,0.04,0.04,0.04,0.04,0.04,0.08,0.16
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.000000e+00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


**Loop over elk punt en bereken de kleinste afstand tot elk centra, \
vervolgens assign elk punt naar een centra**

In [81]:

#src_index = index van huidige rij
for src_index, _ in df.iterrows():
    euclidian_distances = dict()

    #bereken afstanden tot elk centrum
    for centrum_index, _ in kmeans_centra.iterrows():

        #som van kwadraten van de verschillen tussen waarden en centrum
        euclidian_sum = 0 

        for column_name in kmeans_centra.columns:
            current_difference = df.at[src_index, column_name] - kmeans_centra.at[centrum_index, column_name]
            euclidian_sum += current_difference ** 2

        #bereken afstand door wortel van euclidian_sum
        euclidian_distance = math.sqrt(euclidian_sum)

        #voeg euclidian distance toe aan dictionary met centrum index als key
        euclidian_distances[centrum_index] = euclidian_distance

    #Na berekenen afstand data en centrum, zoeken we de kleinste afstand voor elk src_index
    centrum_number = min(euclidian_distances, key = euclidian_distances.get)
    df.at[src_index, 'Centrum'] = centrum_number

df
 


,ADDRESS1_10032 NW 186th,ADDRESS1_1288 Dorchester Avenue,ADDRESS1_1288 South Barrington Ave.,"ADDRESS1_154, Choung-Cheng South Road",ADDRESS1_202-2-3 Hyakunincho,ADDRESS1_2315 Queen's Ave,"ADDRESS1_234-12, Kongdeok-Dong",ADDRESS1_299 Yale Avenue,ADDRESS1_3 Albany Court,ADDRESS1_543-225 Asahi,...,COUNTRY_EN_Korea,COUNTRY_EN_Mexico,COUNTRY_EN_Netherlands,COUNTRY_EN_Spain,COUNTRY_EN_Sweden,COUNTRY_EN_Switzerland,COUNTRY_EN_Taiwan,COUNTRY_EN_United Kingdom,COUNTRY_EN_United States,Centrum
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,1
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,2
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,2
5,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,1
6,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,1
7,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,1
8,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,1
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1


**De juiste centra toewijzen aan rijen uit dataset**


In [82]:
df.groupby('Centrum', as_index = False)['Centrum'].count()

,Centrum
0,1
1,25
2,2


### Evalueren

**intra_cluster_distance \
\
Hoe kleiner hoe beter**

In [83]:
intra_cluster_distances = kmeans.inertia_
print(f'De intra_cluster_afstand is : {intra_cluster_distances}')

De intra_cluster_afstand is : 93.4


**inter_cluster_distances \
\
Hoe groter hoe beter**

In [84]:
#Bereken eerst de euclidische afstand tussen centra van de alle clusters 
#door een dubbele for loop te gebruiken met center_i en center_j 
#overweeg je elk mogelijk paar van clustercentra
 

inter_cluster_distances = dict()

for i, center_i in kmeans_centra.iterrows():
    for j, center_j in kmeans_centra.iterrows():
        if i < j:  # Vermijd herhaling en zelfvergelijking

            # berken de euclidian distance tussen centrum i en centrum j
            distance = np.linalg.norm(center_i - center_j)

            # opslaan distance in dictionary

            #f'{i} - {j}' is sleutel voor indices van de twee centra waarvoor je distance berekend
            #bijvoorbeeld 0 - 1 en 0 -2
            inter_cluster_distances[f'{i} - {j}'] = distance

# Print inter_cluster_distances
for pair, distance in inter_cluster_distances.items():
    print(f"Afstand tussen centrum {pair}: {distance}")

Afstand tussen centrum 0 - 1: 2.2768399153212333
Afstand tussen centrum 0 - 2: 2.7386127875258306
Afstand tussen centrum 1 - 2: 1.6382917933017915


## Clusteringmodel bouwen met meerdere clusters

**Train de model, en geef array met cluster-indelingen**

In [85]:
kmeans = KMeans(n_clusters = 20, random_state= 42) 
prediction_results = kmeans.fit_predict(df)
prediction_results 

array([ 3,  3,  3,  2,  2, 15, 15,  4, 15, 13,  0,  5,  6, 17, 11, 12,  3,
        7, 18,  8, 10, 16, 19, 14,  3, 13,  1,  9])

In [86]:
df['Centrum'] = prediction_results
df

,ADDRESS1_10032 NW 186th,ADDRESS1_1288 Dorchester Avenue,ADDRESS1_1288 South Barrington Ave.,"ADDRESS1_154, Choung-Cheng South Road",ADDRESS1_202-2-3 Hyakunincho,ADDRESS1_2315 Queen's Ave,"ADDRESS1_234-12, Kongdeok-Dong",ADDRESS1_299 Yale Avenue,ADDRESS1_3 Albany Court,ADDRESS1_543-225 Asahi,...,COUNTRY_EN_Korea,COUNTRY_EN_Mexico,COUNTRY_EN_Netherlands,COUNTRY_EN_Spain,COUNTRY_EN_Sweden,COUNTRY_EN_Switzerland,COUNTRY_EN_Taiwan,COUNTRY_EN_United Kingdom,COUNTRY_EN_United States,Centrum
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,3
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,3
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,3
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,2
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,2
5,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,15
6,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,15
7,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,4
8,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,15
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,13


**De juiste centra toewijzen aan rijen uit dataset**

In [87]:
df.groupby('Centrum', as_index = False)['Centrum'].count()

,Centrum
0,1
1,1
2,2
3,5
4,1
5,1
6,1
7,1
8,1
9,1


### Evalueren

**intra_cluster_distance \
\
Hoe kleiner hoe beter**

In [88]:
intra_cluster_distances = kmeans.inertia_
print(f'De intra_cluster_afstand is : {intra_cluster_distances}')

De intra_cluster_afstand is : 22.600000000000005


**inter_cluster_distances \
\
Hoe groter hoe beter**

In [89]:
#Bereken eerst de euclidische afstand tussen centra van de alle clusters 
#door een dubbele for loop te gebruiken met center_i en center_j 
#overweeg je elk mogelijk paar van clustercentra
 

inter_cluster_distances = dict()

for i, center_i in kmeans_centra.iterrows():
    for j, center_j in kmeans_centra.iterrows():
        if i < j:  # Vermijd herhaling en zelfvergelijking

            # berken de euclidian distance tussen centrum i en centrum j
            distance = np.linalg.norm(center_i - center_j)

            # opslaan distance in dictionary

            #f'{i} - {j}' is sleutel voor indices van de twee centra waarvoor je distance berekend
            #bijvoorbeeld 0 - 1 en 0 -2
            inter_cluster_distances[f'{i} - {j}'] = distance

# Print inter_cluster_distances
for pair, distance in inter_cluster_distances.items():
    print(f"Afstand tussen centrum {pair}: {distance}")

Afstand tussen centrum 0 - 1: 2.2768399153212333
Afstand tussen centrum 0 - 2: 2.7386127875258306
Afstand tussen centrum 1 - 2: 1.6382917933017915
